In [ ]:
# Script for Figure 4b

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import iv
from scipy.integrate import solve_ivp
from ipywidgets import interact, FloatSlider

def f(mu, kappa, k1, k2):
    if kappa < 1e-2:
        return -4 * k1 * k2 * np.sin(2 * mu) / kappa
    elif kappa > 1e3:
        return 2 * k1 * (np.cos(2 * mu) - k2) * np.sin(2 * mu)
    else:
        term1 = 1 - 2 * iv(2, kappa / 2) / ((kappa / 2) * iv(1, kappa / 2))
        term2 = 2 / kappa + iv(2, kappa / 2) / iv(1, kappa / 2)
        return 2 * k1 * (term1 * np.cos(2 * mu) - k2 * term2) * np.sin(2 * mu)

# Function g(mu, kappa) for the system
# This function is used to compute the right-hand side of the system of equations
# It is defined based on the parameters mu, kappa, k1, and k2
# The function handles different ranges of kappa to ensure numerical stability
def g(mu, kappa, k1, k2):
    if kappa < 1e-2:
        return 8 * k1 * k2 * np.cos(2 * mu)
    elif kappa > 1e3:
        return 64 * k1 * (k2 * np.cos(2 * mu) - np.cos(4 * mu)) * kappa
    else:
        I0 = iv(0, kappa / 2)
        I1 = iv(1, kappa / 2)
        I2 = iv(2, kappa / 2)
        numerator = 8 * k1 * (k2 * np.cos(2 * mu) - I2 / I1 * np.cos(4 * mu))
        denominator = (kappa / 2) * (I0 / I1 - I1 / I0) - 1
        return numerator / denominator

# Function h(kappa) for the system
def h(kappa):
    if kappa < 1e-2:
        return 2 * kappa
    elif kappa > 1e3:
        return 8 * kappa**2
    else:
        I0 = iv(0, kappa / 2)
        I1 = iv(1, kappa / 2)
        return 4 / (I0 / I1 - I1 / I0 - 2 / kappa)

# RHS of system
def system(mu, kappa, ε, Θ, k1, k2):
    dmu = ε**2 * f(mu, kappa, k1, k2)
    dkappa = ε**2 * g(mu, kappa, k1, k2) - Θ**2 * h(kappa)
    return dmu, dkappa

def system_rhs(t, y, ε, Θ, k1, k2):
    mu, kappa = y
    dmu = ε**2 * f(mu, kappa, k1, k2)
    dkappa = ε**2 * g(mu, kappa, k1, k2) - Θ**2 * h(kappa)
    return [dmu, dkappa]

# Plot phase portrait
def plot_phase_diagram(ε=1.0, Θ=1.0, k1=1.0, k2=-1.0):
    

    mu_range = np.linspace(0, np.pi, 40)
    kappa_range = np.linspace(0.1, 10.0, 40)
    MU, KAPPA = np.meshgrid(mu_range, kappa_range)

    dMU = np.zeros_like(MU)
    dKAPPA = np.zeros_like(KAPPA)
    dmu = np.zeros_like(MU)
    dkappa = np.zeros_like(MU)
    MAG = np.zeros_like(MU)

    for i in range(MU.shape[0]):
        for j in range(MU.shape[1]):
            dmu[i, j], dkappa[i, j] = system(MU[i, j], KAPPA[i, j], ε, Θ, k1, k2)
            norm = np.sqrt(dmu[i, j]**2 + dkappa[i, j]**2)
            MAG[i, j] = norm + 0.1
            if norm != 0:
                dMU[i, j] = dmu[i, j] / norm
                dKAPPA[i, j] = dkappa[i, j] / norm
    
    plt.figure(figsize=(8, 6))
    
    plt.contour(MU / np.pi, KAPPA, dmu, levels=[0], colors='blue', linewidths=2, linestyles='-')
    plt.contour(MU / np.pi, KAPPA, dkappa, levels=[0], colors='red', linewidths=2, linestyles='--')
    plt.axvline(1, color='blue', linestyle='-', linewidth=3, label=r'$\mu = \pi$')  # Add this line
    plt.xlabel(r"$\mu/\pi$",fontsize = 18)
    plt.ylabel(r"$\kappa$", fontsize = 18)


    plt.grid(True)
    plt.xlim(-1e-3, 1+1e-3)
    plt.ylim(1e-1, 5)
    # Add labels for the four regions
    plt.text(0.5 / np.pi, 4.0, r"$\dot\mu > 0, \dot\kappa < 0$", fontsize=14, color="black")
    plt.text(2.2 / np.pi, 4.0, r"$\dot\mu < 0, \dot\kappa < 0$", fontsize=14, color="black")
    plt.text(0.9 / np.pi, 0.8, r"$\dot\mu > 0, \dot\kappa > 0$", fontsize=14, color="black")
    plt.text(1.7 / np.pi, 0.8, r"$\dot\mu < 0, \dot\kappa > 0$", fontsize=14, color="black")
    plt.savefig("phaseportrait2.png", dpi=300)
    plt.show()

# Interactive sliders
interact(
    plot_phase_diagram,
    ε=FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description="ε"),
    Θ=FloatSlider(value=0.9, min=0.0, max=2.0, step=0.1, description=rf"T"),
    k1=FloatSlider(value=0.4, min=-2.0, max=2.0, step=0.1, description="K₁"),
    k2=FloatSlider(value=-2.0, min=-2.0, max=2.0, step=0.1, description="K₂")
)

interactive(children=(FloatSlider(value=1.0, description='ε', max=2.0, min=0.1), FloatSlider(value=0.9, descri…

<function __main__.plot_phase_diagram(ε=1.0, Θ=1.0, k1=1.0, k2=-1.0)>